## Chaining
We can **chain** transformations and action to create a computation **pipeline**
Suppose we want to compute the sum of the squares
$$ \sum_{i=1}^n x_i^2 $$
where the elements $x_i$ are stored in an RDD.

### Start the `SparkContext`

In [1]:
import numpy as np
from pyspark import SparkContext
sc = SparkContext(master="local[4]")

22/10/09 09:24:12 WARN Utils: Your hostname, Inspiron-5567 resolves to a loopback address: 127.0.1.1; using 192.168.1.72 instead (on interface wlp2s0)
22/10/09 09:24:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/09 09:24:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/09 09:24:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
B=sc.parallelize(np.random.randint(0,10,size=1000))
lst = B.collect()
for i in lst: 
    print(i,end=', ')

3, 3, 5, 1, 3, 0, 5, 3, 6, 0, 8, 7, 8, 2, 4, 5, 2, 2, 5, 9, 3, 6, 4, 1, 5, 7, 4, 7, 5, 0, 9, 9, 2, 9, 8, 4, 9, 7, 4, 2, 9, 1, 8, 4, 6, 8, 3, 1, 0, 2, 6, 9, 7, 1, 1, 9, 9, 0, 5, 3, 7, 0, 7, 2, 8, 5, 3, 3, 3, 8, 2, 3, 1, 1, 0, 0, 4, 1, 5, 2, 1, 8, 8, 7, 3, 1, 6, 8, 3, 5, 9, 7, 5, 4, 7, 2, 1, 5, 5, 3, 7, 9, 4, 7, 4, 9, 8, 3, 1, 5, 8, 2, 3, 6, 7, 0, 7, 6, 2, 7, 6, 1, 0, 2, 0, 1, 3, 2, 4, 1, 4, 2, 4, 4, 9, 6, 7, 8, 9, 0, 8, 5, 7, 9, 9, 3, 5, 8, 2, 2, 8, 9, 4, 8, 9, 3, 7, 6, 3, 8, 5, 2, 2, 4, 1, 9, 3, 6, 7, 9, 2, 8, 3, 3, 0, 7, 9, 3, 6, 0, 3, 2, 7, 7, 4, 6, 3, 3, 7, 1, 4, 8, 6, 5, 1, 2, 4, 8, 8, 1, 6, 5, 4, 5, 0, 7, 7, 8, 5, 8, 0, 6, 6, 9, 6, 2, 9, 0, 9, 8, 8, 3, 4, 9, 1, 9, 5, 7, 1, 4, 5, 0, 9, 9, 0, 6, 5, 8, 2, 0, 6, 6, 7, 5, 0, 0, 0, 0, 2, 7, 7, 5, 3, 4, 6, 0, 8, 9, 3, 2, 3, 7, 0, 6, 3, 7, 5, 8, 0, 8, 0, 5, 0, 1, 0, 3, 9, 2, 1, 4, 1, 5, 4, 7, 1, 8, 2, 2, 4, 9, 1, 5, 1, 8, 9, 1, 3, 2, 6, 3, 1, 9, 9, 1, 5, 1, 0, 5, 2, 8, 1, 8, 5, 7, 9, 2, 7, 4, 2, 4, 2, 9, 0, 2, 9, 7, 4, 6, 6, 6, 4, 7, 9, 2

### Sequential syntax for chaining
Perform assignment after each computation

In [3]:
%%time
Squares=B.map(lambda x:x*x)
summation = Squares.reduce(lambda x,y:x+y)

CPU times: user 11.4 ms, sys: 3.85 ms, total: 15.2 ms
Wall time: 883 ms


In [4]:
print(summation)

27251


### Cascaded syntax for chaining
Combine computations into a single cascaded command

In [5]:
%%time
B.map(lambda x:x*x).reduce(lambda x,y:x+y)

CPU times: user 8.23 ms, sys: 4.14 ms, total: 12.4 ms
Wall time: 133 ms


27251

### Both syntaxes mean exactly the same thing
The only difference:
* In the sequential syntax the intermediate RDD has a name `Squares`
* In the cascaded syntax the intermediate RDD is *anonymous*

The execution is identical!

### Sequential execution
The standard way that the map and reduce are executed is
* perform the map
* store the resulting RDD in memory
* perform the reduce

### Disadvantages of Sequential execution

1. Intermediate result (`Squares`) requires memory space.
2. Two scans of memory (of `B`, then of `Squares`) - double the cache-misses.

### Pipelined execution
Perform the whole computation in a single pass. For each element of **`B`**
1. Compute the square
2. Enter the square as input to the `reduce` operation.

### Advantages of Pipelined execution

1. Less memory required - intermediate result is not stored.
2. Faster - only one pass through the Input RDD.

In [8]:
sc.stop()